In [14]:
from pathlib import Path
import requests
import zipfile
import pandas as pd
from shapely import wkt
import rioxarray as rxr
import geopandas as gpd

In [3]:
storage_path = Path('/storage/dlhogan/summa_modeling_data/prism/')

In [22]:
def download_prism_normals(resolution: str, element: str, date: str, save_path: str):
    """
    Downloads PRISM normals data for the given resolution, element, and date.
    
    Parameters:
        resolution (str): '4km' or '800m'
        element (str): One of ['ppt', 'tmin', 'tmax', 'tmean', 'tdmean', 'vpdmin', 'vpdmax', 'solslope', 'soltotal', 'solclear', 'soltrans']
        date (str): 'MM' for monthly, 'MMDD' for daily, or '14' for annual normal
        save_path (str): Directory where the downloaded file should be saved
    """
    base_url = "https://services.nacse.org/prism/data/public/normals"
    url = f"{base_url}/{resolution}/{element}/{date}"
    
    response = requests.get(url, stream=True)
    if "ppt" not in [f.name for f in (storage_path / "PRISM_800m_annual_normals").glob("*")][0]:
        
        if response.status_code == 200:
            filename = f"PRISM_{resolution}_{element}_{date}.zip"
            filepath = os.path.join(save_path, filename)
            
            with open(filepath, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)
            
            print(f"Download complete: {filepath}")

        else:
            print(f"Failed to download data. HTTP Status Code: {response.status_code}")
            return
    
        unzip_and_move_files(storage_path / f"PRISM_800m_{element}_14.zip", storage_path / "PRISM_800m_annual_normals")
        return
    else:
        print("PRISM ppt normals already downloaded")
        return

# Example usage
# download_prism_normals("800m", "tmean", "0430", "./data")

def unzip_and_move_files(zip_path: str, save_path: str):
    """
    Unzips the file at the given path and moves the contents to the save path.
    
    Parameters:
        zip_path (str): Path to the zip file
        save_path (str): Directory where the contents should be saved
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(save_path)

    # deleter the zip file
    os.remove(zip_path)
    
    print(f"Unzipped and moved files to: {save_path}")
    return

In [24]:
# download PRISM normals
download_prism_normals("800m", "ppt", "14", storage_path)
download_prism_normals("800m", "tmean", "14", storage_path)

PRISM ppt normals already downloaded
PRISM ppt normals already downloaded


In [25]:
ds = rxr.open_rasterio(storage_path / 'PRISM_800m_annual_normals' / 'PRISM_ppt_30yr_normal_800mM4_annual_bil.bil')

In [58]:
basin_name = "EastRiver"
basin_path = Path(f'/storage/dlhogan/summa_modeling_data/domain_{basin_name}')
basin = gpd.read_file(basin_path / f'{basin_name}.shp')
basin = basin.to_crs(ds.rio.crs)

basin_prism = ds.rio.clip(basin.geometry)
mask = basin_prism >= 0 

basin_prism_ma = basin_prism.where(mask)

# save the data
basin_prism_ma.rio.to_raster(basin_path / "forcing" / "01_raw_data" / f'{basin_name}_PRISM_ppt_30yr_normal_800m.tif')


In [59]:
# open data observation
obs_path = Path('/storage/dlhogan/summa_modeling_data/domain_{basin_name}/forcing/01_raw_data')

In [60]:
for f in basin_path.glob('OBS*'):
    # get all characters after OBS
    obs_name = str(f).split('OBS_')[1][:-4]
    # read dataframe and convert to gdf
    df = pd.read_csv(f)
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    # nearest PRISM data location
    nearest_prism = basin_prism_ma.squeeze().sel(x=gdf.geometry.x[0], y=gdf.geometry.y[0], method='nearest')
    # calculate site weights
    site_weights = basin_prism_ma.squeeze()/nearest_prism
    # save weights
    site_weights.rio.to_raster(basin_path / "forcing" / "01_raw_data" / f'{basin_name}_PRISM_ppt_30yr_normal_800m_{obs_name}_weights.tif')